## GUI of The Model ❤

_We have used tkinter for the GUI of model, a simple one to test our model_😊

![dd](https://lithub.com/wp-content/uploads/2019/07/writing.jpg)

### importation des Librairies ✔

In [2]:
# Importer torch
import skimage 
from skimage import morphology
import torch
import skimage
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.linear_model import LinearRegression
from skimage import transform as tf
from sklearn.model_selection import train_test_split

### Feature Extraction & Preprocessing ✔ 

In [3]:


#####  IMAGE TRANSFORMATIONS  #####

def to_binary(img) :
  img = np.array(img, dtype=float)
  for i in range(0, 28):
    for k in range(0, 28):
      if img[i][k] != 0 :
        img[i][k] = 1
  return img
  
def to_thin(img):
  img_bin = to_binary(img)
  img_skl = skimage.morphology.skeletonize(img_bin)
  thin= skimage.morphology.thin(img_skl , max_iter=3)
  return thin

def crop(img):
    img = img[~np.all(img == 0, axis=1)]
    img = np.transpose(img)
    img = img[~np.all(img== 0, axis=1)]
    img = np.transpose(img)
    img = tf.resize(img, (28, 28), order=0)
    return img

def split(img):
	blocks = []
	[row, col] = img.shape
	for i in range (0, row, 7):
		for j in range(0, col, 7):
			blocks.append(img[i:i+7, j:j+7])
	return blocks

#####  BLOCK HELPERS  #####

def black_pix(block):
	S =0
	for i in range(0,7):
		for j in range(0,7):
			if block[i][j] != 0:
				S += 1
	return S / (block.shape[0] * block.shape[1])

def get_coordinates(block):
	X=[]
	Y=[]
	
	for i in range(0, 7):
		for j in range(0, 7):
			if block[i][j] != 0:
				Y.append(i)
				X.append(j)
	return X,Y

#####  FEATURE EXTRACTORS  #####

def get_features(block) : 
	one = black_pix(block)
	two = 0
	three = 0
	
	X, Y = get_coordinates(block)
		
	if len(X) > 0 :
		X = np.array(X).reshape((-1, 1))
		Y = np.array(Y)
		reg = LinearRegression()
		reg.fit(X,Y)
		a=reg.coef_
		
		two   = 2 * a / (1 + a ** 2)
		three = (1 - a ** 2) / (1 + a ** 2) 

	return float(one), float(two), float(three)

def show_slopes(img):
	img = to_thin(img)
	img = crop(img)
	blocks = split(img)

	figure = plt.figure(figsize=(10, 14))
	for index in range(1, 17):
		  plt.subplot(4, 4, index)
		  blck = blocks[index-1]
		  X, Y = get_coordinates(blck)
		  a = -1
		  b = 0
		  if len(X) > 0 :
		    X = np.array(X).reshape((-1, 1))
		    Y = np.array(Y)
		    reg = LinearRegression()
		    reg.fit(X,Y)
		    a = reg.coef_
		    b = reg.intercept_
		  plt.title("{:.2f}".format(float(a)))
		  plt.imshow(blck, cmap='gray')
		  if len(X) > 0 :
		    plt.plot([0, 6], [b, a*7+b])
	plt.show()

def preprocess(img):
	img = to_thin(img)
	img = crop(img)
	blocks = split(img)
	
	ins = []
	
	for blck in blocks:
		one, two, three = get_features(blck)
		ins.append(one)
		ins.append(two)
		ins.append(three)
		
	return ins


### Notre NN ✔

In [4]:
# Importer le module nn
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net (nn.Module):
    def __init__ (self):
        
        super(Net, self).__init__()
        self.fc1 = nn.Linear(48, 200)
        nn.BatchNorm1d(200)
        nn.Dropout(0.5)
        self.fc2 = nn.Linear(200, 100)
        nn.BatchNorm1d(100)
        nn.Dropout(0.5)
        self.fc4= nn.Linear(100, 20)
        nn.BatchNorm1d(20)
        nn.Dropout(0.5)
        self.fc5= nn.Linear(20, 20)
        nn.BatchNorm1d(20)
        nn.Dropout(0.5)
        self.fc3= nn.Linear(20, 10)

    def forward(self, x):
        x = F.selu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.selu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc3(x)
        return x

### Here we go with Tkinter ❤

In [24]:
from tkinter import *

import cv2
import numpy as np
from PIL import ImageGrab, Image, ImageDraw
import torch


model= Net()
model.load_state_dict(torch.load("model_final_cas6.pth"))
model.eval()
print("successfully")

image_folder = "C:/Users/OMAIMA/Desktop/mnist"
print("successfully")

# canvas
root = Tk()
root.resizable(0, 0)
root.title("I can guess your handwritten digits 🖌 ")

width  = 600
height = 400
bg     = (255, 255, 255)

lastx, lasty = None, None
image_number = 0
image        = Image.new("RGB", (width, height), bg)
draw         = ImageDraw.Draw(image)

canvas = Canvas(root, width=width, height=height, bg="white")
canvas.grid(row=0, column=0, pady=2, sticky=NSEW, columnspan=2)

def activate_event(event):
    global lastx, lasty
    canvas.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event.y
def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event.y
    canvas.create_line((lastx, lasty, x, y), width=8, fill='black', capstyle=ROUND, smooth=TRUE, splinesteps=12)
    draw.line([lastx, lasty, x, y], fill='black', width=8)
    lastx, lasty = x, y
canvas.bind('<Button-1>', activate_event)


def clear_widget():
    global canvas, image, draw
    canvas.delete('all')
    image = Image.new("RGB", (width, height), bg)
    draw = ImageDraw.Draw(image)
button_clear = Button(text='Clear Output',width=15, height=3, command=clear_widget)
button_clear.grid(row=2, column=1, pady=1, padx=1)

def Recognize_Digit():
	global image_number
	global canvas
	filename = f'img_{image_number}.png'
    
	x = root.winfo_rootx() + canvas.winfo_x()
	y = root.winfo_rooty() + canvas.winfo_y()
	x1 = x + canvas.winfo_width()
	y1 = y + canvas.winfo_height()
	print(x, y, x1, y1)
    
	image.save(image_folder + filename)
	img = cv2.imread(image_folder + filename, cv2.IMREAD_COLOR)
	gray = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)
	ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

	contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
	for cnt in contours:
		x, y, w, h = cv2.boundingRect(cnt)
		# make a rectangle box around each curve
		cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 1)

		# Cropping out the digit from the image corresponding to the current contours in the for loop
		digit = th[y:y + h, x:x + w]

		# Resizing that digit to (18, 18)
		resized_digit = cv2.resize(digit, (18, 18))

		# Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
		padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)

		digit = padded_digit.reshape(28, 28)
		ins = preprocess(digit)
		ins= torch.tensor(ins).reshape(1,48)
		pred = model(ins.float())
		final_pred = torch.argmax(pred)

		data = str(final_pred)

		font = cv2.FONT_HERSHEY_SIMPLEX
		fontScale = 0.5
		color = (255, 0, 0)
		thickness = 1
		cv2.putText(img, data, (x, y - 5), font, fontScale, color, thickness)

	cv2.imshow('Prédictions', img)
	cv2.waitKey(10000) 

btn_rec = Button(text='Recognize Digits',width=15, height=3, command=Recognize_Digit)
btn_rec.grid(row=2, column=0, pady=1, padx=1)


root.mainloop()


successfully
successfully
34 99 638 503
34 99 638 503
34 99 638 503
34 99 638 503


### Made with ❤❤❤